In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.openai_functions import create_extraction_chain, create_tagging_chain
from langchain.prompts import ChatPromptTemplate

In [3]:
llm=ChatOpenAI(temperature=0, 
                   model="gpt-3.5-turbo-0613")

## Tagging chain

In [4]:
schema = {
      "properties": {
          "person_name": {"type": ["string"]}, 
          "person_height":{"type": ["integer"]},
          "person_hair_color": {"type": ["string"]},
          "dog_name": {"type": ["string"]},
          "dog_breed": {"type": ["string"]}
              },
        "required": ["person_name", "height"]
    }

In [5]:
chain = create_tagging_chain(schema=schema, llm=llm)

In [6]:
user_inp = """
Alex is 5 feet long. Claudia is 4 feet than Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.
Alex's dog Frosty is a labrador and likes to play hide and seek.
        """

In [7]:
chain(user_inp)

{'input': "\nAlex is 5 feet long. Claudia is 4 feet than Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.\nAlex's dog Frosty is a labrador and likes to play hide and seek.\n        ",
 'output': {'person_name': ['Alex', 'Claudia'],
  'person_height': ['5 feet', '4 feet'],
  'person_hair_color': ['blonde', 'brunette'],
  'dog_name': ['Frosty'],
  'dog_breed': ['labrador']}}

## Extracting chain

1. Establishing range and options

In [57]:
schema = {
      "properties": {
          "sentiment": {"type": "string", "enum": ["happy", "neutral", "sad"], "description": "sentiment of the statement"}, 
          "aggressiveness": {"type": "integer", "enum": [1,2,3,4,5]},
          "language": {"type": "string"},
              },
        "required": []
    }

In [58]:
user_inp = """
Estoy tan contento de haberte conocido! Creo que seremos muy buenos amigos!
        """

In [59]:
chain = create_extraction_chain(schema=schema, llm=llm)

In [60]:
chain(user_inp)

{'input': '\nEstoy tan contento de haberte conocido! Creo que seremos muy buenos amigos!\n        ',
 'output': {'sentiment': 'super happy',
  'aggressiveness-0_to_10': 0,
  'language': 'Spanish'}}

In [61]:
user_inp = """
Estoy muy enojado con vos! Te voy a dar tu merecido!
"""

In [62]:
chain = create_extraction_chain(schema=schema, llm=llm)

In [63]:
chain(user_inp)

{'input': '\nEstoy muy enojado con vos! Te voy a dar tu merecido!\n',
 'output': {'sentiment': 'sad',
  'aggressiveness-0_to_10': 8,
  'language': 'Spanish'}}